# Reading images using BioIO

As mentioned in a previous section, using the [scikit-image](https://scikit-image.org/) offers the ability to read images in some formats but it has its limitations in terms of the formats it can read or how it can handle metadata e.g. the dimension order of an image with more than three dimensions.

[BioIO](https://github.com/bioio-devs/bioio) can read many various image formats e.g. OME-TIFF, PNG, ND2, but **only** when paired with the appropriate plug-in for the image format. It also offers a bridge with the Bio-Formats library. 

## Launch
This notebook uses the [environment_python.yml](./envs/environment_python.yml) file. See [Set up environment](SoftwarePackagesIntro.ipynb) for more details.


## Reading nd2 

When working with microscopy image data, many file formats are used such as the nd2 format. In this notebook, we will open a Nikon NIS-Elements ND2 (extension``.nd2``) file.

First we need to install the BioIO plug-in to read the ``.nd2`` format.

We will in following cells install [bioio-nd2](https://pypi.org/project/bioio-nd2/) plugin from [PyPI](https://pypi.org).

### Read the file

We will download the file from the publicly available data samples and read it using BioIO library.

Run the following cell only when in Google Colab.

In [ ]:
! pip install bioio==1.2.0 scikit-image==0.22.0

In [ ]:
!wget /content/sample_data/test.nd2 https://downloads.openmicroscopy.org/images/ND2/aryeh/MeOh_high_fluo_011.nd2

In [ ]:
from bioio import BioImage
import bioio_nd2

img = BioImage("/content/MeOh_high_fluo_011.nd2", reader=bioio_nd2.Reader)

Return the dimension order. This information **woul not be** available when using scikit-image only.

In [4]:
img.dims.order

'TCZYX'

These formats usually have extra dimensions to accomodate many different data shapes. This is their common order when working with nd2 files:

* 'C': Channel or wavelength
* 'T': Time
* 'X': Width
* 'Y': Height
* 'Z': Slice (depth) or number of Z-sections
* 'R': Rotation
* 'S': Scene: contiguous regions of interest in a mosaic image

In [5]:
# Return size of X dimension 
img.dims.X

32751

In [6]:
# Return the name of the channels
img.channel_names

['EGFP']

In [7]:
# Return the physical size X
img.physical_pixel_sizes.X 

0.3444225755520869

### Exercise 1:
 * Check the size and physical size along Y.

In [7]:
# Return the size of the numpy array
img.shape

(1, 1, 1, 20684, 32751)

In [8]:
# Access the 5D numpy array
img.data

array([[[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]]], dtype=uint16)

In [17]:
# Return 4D CZYX numpy array
data = img.get_image_data("CZYX", T=0)
data.shape

# Return 3D ZYX numpy array
data = img.get_image_data("ZYX", T=0, C=0)
data.shape

(1, 1, 20684, 32751)

### Exercise 2:
* Create a 2D YX numpy array and display it using the matplotlib library

You can find more information about the CZI file

In [12]:
# Get the id of the current operating scene. In our case we only have one scene
img.current_scene

# Get a list valid scene ids
img.scenes

('ScanRegion0',)

## Save the image

When analysing data, it is often useful to save the result image.
You usually cannot save it in the original format.

* saving the result as a ``png`` or ``jpeg`` is only useful if you wish to insert the image in a presentation for example.
* If you wish to re-use the result image, we recommend to save it in the ``OME-TIFF`` format.
  
The BioIO library has a method to the save the image as OME-TIFF.

In [ ]:
# Save the image as OME-TIFF
img.save("MeOh_high_fluo_011.nd2.ome.tiff")

### Exercise 3:

* Install the required dependencies in the environment to read a ``.lif`` file.
* Read the ``lif`` image from [imagesc-30856](https://downloads.openmicroscopy.org/images/Leica-LIF/imagesc-30856/).
* Print out the shape of the array and the dimensions order
* Display the first 2D-plane using the matplotlib package

## Common problem
It is common to try to view the data in a given library and the file is not supported. In that case, you need to think about converting the data to be able to do so.